In [0]:
from pyspark.sql.functions import col
import tensorflow as tf
spark_df = spark.read.format("delta").load("/databricks-datasets/flowers/delta") \
  .select(col("content"), col("label_index")) \
  .limit(100)


In [0]:
path = '/ml/flowersData/converted_data.tfrecord'
spark_df.write.format("tfrecords").mode("overwrite").save(path)
display(dbutils.fs.ls(path))

path,name,size
dbfs:/ml/flowersData/converted_data.tfrecord/_SUCCESS,_SUCCESS,0
dbfs:/ml/flowersData/converted_data.tfrecord/part-r-00000,part-r-00000,17175166


In [0]:
import os
filenames = [(f"/dbfs{path}/{name}") for name in os.listdir("/dbfs" + path) if name.startswith("part")]
dataset = tf.data.TFRecordDataset(filenames)


In [0]:
def decode_and_normalize(serialized_example, image_size = 224):  
  # Parse from single example
  feature_dataset = tf.io.parse_single_example(
      serialized_example,
      features={
          'content': tf.io.FixedLenFeature([], tf.string),
          'label_index': tf.io.FixedLenFeature([], tf.int64),
      })
  # Decode the parsed data
  image = tf.io.decode_jpeg(feature_dataset['content'])
  label = tf.cast(feature_dataset['label_index'], tf.int32)
  # Resize the decoded data into the desired size
  image = tf.image.resize(image, [image_size, image_size])
  # Finally, normalize the data
  image = tf.cast(image, tf.float32) * (1. / 255) - 0.5
  return image, label

In [0]:
parsed_dataset = dataset.map(decode_and_normalize)

In [0]:
parsed_dataset

Out[6]: <MapDataset shapes: ((224, 224, None), ()), types: (tf.float32, tf.int32)>

In [0]:
from pyspark.sql.types import *
path = "test-output.tfrecord"
fields = [StructField("id", IntegerType()), 
StructField("IntegerCol", IntegerType()),
StructField("LongCol", LongType()), 
StructField("FloatCol", FloatType()),
StructField("DoubleCol", DoubleType()), 
StructField("VectorCol", ArrayType(DoubleType(), True)),
StructField("StringCol", StringType())]
schema = StructType(fields)

test_rows = [[11, 1, 23, 10.0, 14.0, [1.0, 2.0], "r1"], [21, 2, 24, 12.0, 15.0, [2.0, 2.0], "r2"]]
rdd = spark.sparkContext.parallelize(test_rows)
df = spark.createDataFrame(rdd, schema)
path= 'dbfs:/tmp/dataset'
df.write.format("tfrecords").option("recordType", "Example").save(path)
display(df)

id,IntegerCol,LongCol,FloatCol,DoubleCol,VectorCol,StringCol
11,1,23,10.0,14.0,"List(1.0, 2.0)",r1
21,2,24,12.0,15.0,"List(2.0, 2.0)",r2


In [0]:
df = spark.read.format("tfrecords").option("recordType", "Example").load(path)
df.show()


In [0]:
import os
import subprocess
import uuid
work_dir = os.path.join("/ml/tmp/petastorm", str(uuid.uuid4()))
dbutils.fs.mkdirs(work_dir)
def get_local_path(dbfs_path):
  return os.path.join("/dbfs", dbfs_path.lstrip("/"))


In [0]:
data_url = "https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/multiclass/mnist.bz2"
libsvm_path = os.path.join(work_dir, "mnist.bz2")
subprocess.check_output(["wget", data_url, "-O", get_local_path(libsvm_path)])
df = spark.read.format("libsvm") \
 .option("numFeatures", "784") \
 .load(libsvm_path)


In [0]:
%scala
import org.apache.spark.ml.linalg.Vector
val toArray = udf {v: Vector => v.toArray }
spark.sqlContext.udf.register("toArray", toArray)

import org.apache.spark.ml.linalg.Vector
toArray: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$7348/602836013@1855f637,ArrayType(DoubleType,false),List(Some(class[value[0]: vector])),None,true,true)
res1: org.apache.spark.sql.expressions.UserDefinedFunction = SparkUserDefinedFunction($Lambda$7348/602836013@1855f637,ArrayType(DoubleType,false),List(Some(class[value[0]: vector])),None,true,true)

In [0]:
parquet_path = os.path.join(work_dir, "parquet")
df.selectExpr("toArray(features) AS features", "int(label) AS label") \
 .repartition(10) \
 .write.mode("overwrite") \
 .option("parquet.block.size", 1024 * 1024) \
 .parquet(parquet_path)

In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import models, layers
from petastorm import make_batch_reader
from petastorm.tf_utils import make_petastorm_dataset
def get_model():
  model = models.Sequential()
  model.add(layers.Conv2D(32, kernel_size=(3, 3),
                         activation='relu',
                         input_shape=(28, 28, 1)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Dropout(0.25))
  model.add(layers.Flatten())
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(10, activation='softmax'))
  return model


In [0]:
petastorm_dataset_url = "file://" + get_local_path(parquet_path)

In [0]:
with make_batch_reader(petastorm_dataset_url, num_epochs=100) as reader:
  dataset = make_petastorm_dataset(reader) \
  .map(lambda x: (tf.reshape(x.features, [-1, 28, 28, 1]), tf.one_hot(x.label, 10)))
  model = get_model()
  optimizer = keras.optimizers.Adadelta()
  model.compile(optimizer=optimizer,
               loss='categorical_crossentropy',
               metrics=['accuracy'])
  model.fit(dataset, steps_per_epoch=10, epochs=10)


WARNING:tensorflow:AutoGraph could not transform <function <lambda> at 0x7fba4c456cb0> and will run it as-is.
Cause: could not parse the source code:

 .map(lambda x: (tf.reshape(x.features, [-1, 28, 28, 1]), tf.one_hot(x.label, 10)))

This error may be avoided by creating the lambda in a standalone statement.

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
WARNING: AutoGraph could not transform <function <lambda> at 0x7fba4c456cb0> and will run it as-is.
Cause: could not parse the source code:

 .map(lambda x: (tf.reshape(x.features, [-1, 28, 28, 1]), tf.one_hot(x.label, 10)))

This error may be avoided by creating the lambda in a standalone statement.

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Epoch 1/10
/databricks/python/lib/python3.7/site-packages/petastorm/arrow_reader_worker.py:53: FutureWarning: Calling .data on ChunkedArray is provided for compatibility after Column was removed, simply drop this attribute
 column_as_pandas = column.data.chunks[0].to_pandas()
 1/10 [==>...........................] - ETA: 0s - loss: 48.1649 - accuracy: 0.1091 2/10 [=====>........................] - ETA: 0s - loss: 49.2924 - accuracy: 0.1182 3/10 [========>.....................] - ETA: 0s - loss: 51.4696 - accuracy: 0.1091 4/10 [===========>..................] - ETA: 0s - loss: 51.0481 - accuracy: 0.1015 5/10 [==============>...............] - ETA: 0s - loss: 51.2059 - accuracy: 0.0945 6/10 [=================>............] - ETA: 0s - loss: 51.4071 - accuracy: 0.0879 7/10 [====================>.........] - ETA: 0s - loss: 50.8081 - accuracy: 0.0848 8/10 [=======================>......] - ETA: 0s - loss: 50.2503 - accuracy: 0.0864 9/10 [==========================>...] - ETA: 0s - loss: 50.0768 - accuracy: 0.087510/10 [==============================] - ETA: 0s - loss: 49.8006 - accuracy: 0.088510/10 [==============================] - 1s 149ms/step - loss: 49.8006 - accuracy: 0.0885
Epoch 2/10
 1/10 [==>...........................] - ETA: 0s - loss: 49.0300 - accuracy: 0.1030 2/10 [=====>........................] - ETA: 0s - loss: 49.7267 - accuracy: 0.1061 3/10 [========>.....................] - ETA: 0s - loss: 47.7714 - accuracy: 0.1111 4/10 [===========>..................] - ETA: 0s - loss: 47.8977 - accuracy: 0.1030 5/10 [==============>...............] - ETA: 0s - loss: 47.3492 - accuracy: 0.1042 6/10 [=================>............] - ETA: 0s - loss: 46.8213 - accuracy: 0.1071 7/10 [====================>.........] - ETA: 0s - loss: 46.9480 - accuracy: 0.1022

In [0]:
import pandas as pd
from PIL import Image
import numpy as np
import io
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from pyspark.sql.functions import col, pandas_udf, PandasUDFType

images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load("/databricks-datasets/flower_photos")

display(images.limit(5))

path,modificationTime,length,content
dbfs:/databricks-datasets/flower_photos/tulips/2431737309_1468526f8b.jpg,2019-12-11T22:18:32.000+0000,281953,/9j/4AAQSkZJRgABAQEBLAEsAAD/4gxYSUNDX1BST0ZJTEUAAQEAAAxITGlubwIQAABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAAAAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAA= (truncated)
dbfs:/databricks-datasets/flower_photos/sunflowers/4932735362_6e1017140f.jpg,2019-12-11T22:18:00.000+0000,277326,/9j/4AAQSkZJRgABAQEASABIAAD/2wBDAAEBAQEBAQEBAQEBAQECAgMCAgICAgQDAwIDBQQFBQUEBAQFBgcGBQUHBgQEBgkGBwgICAgIBQYJCgkICgcICAj/2wBDAQEBAQICAgQCAgQIBQQFCAgICAgICAgICAgICAgICAgICAg= (truncated)
dbfs:/databricks-datasets/flower_photos/tulips/8717900362_2aa508e9e5.jpg,2019-12-11T22:18:52.000+0000,265806,/9j/4AAQSkZJRgABAQEASABIAAD/4gxYSUNDX1BST0ZJTEUAAQEAAAxITGlubwIQAABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAAAAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAA= (truncated)
dbfs:/databricks-datasets/flower_photos/sunflowers/4341530649_c17bbc5d01.jpg,2019-12-11T22:17:56.000+0000,257418,/9j/4AAQSkZJRgABAQEASABIAAD/4gxYSUNDX1BST0ZJTEUAAQEAAAxITGlubwIQAABtbnRyUkdCIFhZWiAHzgACAAkABgAxAABhY3NwTVNGVAAAAABJRUMgc1JHQgAAAAAAAAAAAAAAAAAA9tYAAQAAAADTLUhQICAAAAAAAAA= (truncated)
dbfs:/databricks-datasets/flower_photos/daisy/5693459303_e61d9a9533.jpg,2019-12-11T22:16:30.000+0000,248540,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAEBAQEBAQEBAQEBAQECAgMCAgICAgQDAwIDBQQFBQUEBAQFBgcGBQUHBgQEBgkGBwgICAgIBQYJCgkICgcICAj/2wBDAQEBAQICAgQCAgQIBQQFCAgICAgICAgICAgICAgICAgICAg= (truncated)


In [0]:
model = ResNet50(include_top=False)
model.summary()  # verify that the top layer is removed
bc_model_weights = sc.broadcast(model.get_weights())

def model_fn():
  model = ResNet50(weights=None, include_top=False)
  model.set_weights(bc_model_weights.value)
  return model

def preprocess(content):
  img = Image.open(io.BytesIO(content)).resize([224, 224])
  arr = img_to_array(img)
  return preprocess_input(arr)

def featurize_series(model, content_series):
  input = np.stack(content_series.map(preprocess))
  preds = model.predict(input)
  output = [p.flatten() for p in preds]
  return pd.Series(output)


Downloading data from https://storage.googleapis.com/tensorflow/keras-applications/resnet/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5
 8192/94765736 [..............................] - ETA: 1s 4202496/94765736 [>.............................] - ETA: 1s16637952/94765736 [====>.........................] - ETA: 0s25174016/94765736 [======>.......................] - ETA: 0s34684928/94765736 [=========>....................] - ETA: 0s44761088/94765736 [=============>................] - ETA: 0s54894592/94765736 [================>.............] - ETA: 0s65314816/94765736 [===================>..........] - ETA: 0s75505664/94765736 [======================>.......] - ETA: 0s83795968/94765736 [=========================>....] - ETA: 0s94773248/94765736 [==============================] - 1s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type) Output Shape Param # Connected to 
==================================================================================================
input_1 (InputLayer) [(None, None, None, 0 
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D) (None, None, None, 3 0 input_1[0][0] 
__________________________________________________________________________________________________
conv1_conv (Conv2D) (None, None, None, 6 9472 conv1_pad[0][0] 
__________________________________________________________________________________________________
conv1_bn (BatchNormalization) (None, None, None, 6 256 conv1_conv[0][0] 
__________________________________________________________________________________________________
conv1_relu (Activation) (None, None, None, 6 0 conv1_bn[0][0] 
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D) (None, None, None, 6 0 conv1_relu[0][0] 
__________________________________________________________________________________________________
pool1_pool (MaxPooling2D) (None, None, None, 6 0 pool1_pad[0][0] 
__________________________________________________________________________________________________
conv2_block1_1_conv (Conv2D) (None, None, None, 6 4160 pool1_pool[0][0] 
__________________________________________________________________________________________________
conv2_block1_1_bn (BatchNormali (None, None, None, 6 256 conv2_block1_1_conv[0][0] 
__________________________________________________________________________________________________
conv2_block1_1_relu (Activation (None, None, None, 6 0 conv2_block1_1_bn[0][0] 
__________________________________________________________________________________________________
conv2_block1_2_conv (Conv2D) (None, None, None, 6 36928 conv2_block1_1_relu[0][0] 
__________________________________________________________________________________________________
conv2_block1_2_bn (BatchNormali (None, None, None, 6 256 conv2_block1_2_conv[0][0] 
__________________________________________________________________________________________________
conv2_block1_2_relu (Activation (None, None, None, 6 0 conv2_block1_2_bn[0][0] 
__________________________________________________________________________________________________
conv2_block1_0_conv (Conv2D) (None, None, None, 2 16640 pool1_pool[0][0] 
___________________________________

In [0]:
@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
  model = model_fn()
  for content_series in content_series_iter:
    yield featurize_series(model, content_series)

/databricks/spark/python/pyspark/sql/pandas/functions.py:386: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
 "in the future releases. See SPARK-28264 for more details.", UserWarning)

In [0]:
spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")
features_df = images.repartition(16).select(col("path"), featurize_udf("content").alias("features"))
features_df.write.mode("overwrite").parquet("dbfs:/ml/tmp/flower_photos_features")
